# **Signa_Lab ITESO:** Postprocesador de datos de redes sociodigitales
## **Cuaderno: Meta AdLibrary (listas de anuncios)**

Cuaderno de código abierto diseñado para importar datos sobre listas de las plataforma de Meta (Facebook, Instagram, Messenger, Audience Network) descargados de la plataforma Meta AdLibrary, y preparar sus datos para su posterior análisis exploratorio. Este cuaderno aporta también un análisis desagregado de las estrategias de segmentación y alcance aplicadas para cada anuncio y transparentadas en Meta*.

**\***Los grupos de celdas marcadas con **asterisco requieren información** antes de seguir adelante, que podrás agregas en los campos a llenar que desplieguen por esas celdas.

## **0. Introducción**

Antes de comenzar, necesitas obtener al menos un archivo en formato **CSV o Excel** que registre **listas de anuncios** y sus metadatos, descargables de Meta AdLibrary.

Puedes leer la documentación de la [plataforma web](https://www.facebook.com/ads/library/) y de [la API](https://www.facebook.com/ads/library/api/) para saber más sobre los datos consultables y cómo interpretar los campos que arrojan.

La [plataforma web MetaAdLibrary](https://www.facebook.com/ads/library/) permite descargar datos de anuncios como archivo CSV.

Se pueden hacer **hasta 3 descargas por día**, exclusivamente sobre anuncios marcados por Meta como ***Temas sociales, elecciones o política***.

Permite **formular búsquedas**, opcionalmente acotadasa a algún país, por:
* Páginas de anunciantes específicos
* Anuncios relevantes por palabras clave o frases exactas
* Anuncios segmentados a cierta ubicación (*Región* o *Ciudad*)

A su vez se pueden **filtrar los resultados** de tu búsqueda por parámetros como:
* Idioma
* Anunciante (lista completa de anunciantes en los resultados de la búsqueda)
* Plataforma (*Facebook*, *Instagram*, *Audience Network*, *Messenger*)
* Tipo de contenido multimedia (*Imágenes*, *Memes*, *Memes y videos*, *Videos*, *Sin imagen ni video*)
* Estado (*Activos e inactivos*, *Anuncios activos*, *Anuncios inactivos*)
* Impresiones por fecha (intervalo por fechas exactas)

Para poder descargar los archivos en CSV, es necesario haber iniciado sesión, aunque la plataforma se pueden explorar resultados de búsqueda sin haberlo hecho.


## **1. Importar librerías y archivos de datos**

### Instalar e importar librerías:

In [ ]:
# Instalar librerías de Python necesarias
!pip install pandas
!pip install matplotlib
!pip install simpledbf
!pip install nltk
!pip install folium pandas
!pip install unidecode

In [ ]:
# Importar librerías de Python necesarias
import pandas as pd
import copy as copy
import matplotlib.pyplot as plt
import re
import simpledbf

from IPython.display import display, clear_output
import ipywidgets as widgets

import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')

import plotly.express as px
import geopandas as gpd
from simpledbf import Dbf5
import seaborn as sns

import unidecode
import folium
import requests

### *Indicar rutas de archivos de datos a importar y nombre de proyecto:

**Importar y concatenar archivos de datos** (en CSV o Excel):

In [ ]:
# Definir función para cargar archivos a partir de la extensión en su ruta indicada
def load_file(path):
    if path.endswith('.csv'):
        return pd.read_csv(path)
    elif path.endswith('.xlsx'):
        return pd.read_excel(path)
    else:
        raise ValueError("Formato no compatible. Por favor carga solo archivos .csv or .xlsx.")

# Inicializar lista para alojar todas las rutas y una variable para el DataFrame final, accesible globalmente
file_paths = []
dfs = []
df = None  # DataFrame global

# Definir función para añadir un nuevo campo de texto (input) para añadir una ruta de archivo adicional
def add_file_input(b=None):
    path_input = widgets.Text(value='', placeholder='Escribe la ruta del archivo', description=f'Archivo {len(file_paths) + 1}:')
    file_paths.append(path_input)
    update_ui()

# Definir función para eliminar el último campo de texto (input) para ruta de archivo
def remove_file_input(b=None):
    if file_paths:
        file_paths.pop()
        update_ui()

# Definir función para procesar y cargar todos los archivos
def process_files(b):
    global dfs, df
    dfs = []  # Vaciar DataFrames

    for path_input in file_paths:
        path = path_input.value
        try:
            temp_df = load_file(path)
            temp_df['filename'] = path  # Add a column with the filename
            dfs.append(temp_df)
            print(f"Nombre de archivo: {path}")
            print(f"Filas/Columnas (shape): {temp_df.shape}")
        except ValueError as e:
            print(f"Error al cargar el archivo {path}: {e}")
            return

    if dfs:
        df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
        print("\n¡Se cargaron todos los archivos!")
        print(f"Filas/Columnas (shape) de DataFrame creado: {df.shape}")

# Campo de texto (input) para indicar nombre del proyecto (para integrarse en nombres de archivos a exportar)
project_name = widgets.Text(value='', placeholder='Escribe el nombre del proyecto (corto y sin espacios)', description='Nombre proyecto:')

# Botones para añadir y eliminar archivos
add_button = widgets.Button(description="Añadir archivo",  button_style='')
remove_button = widgets.Button(description="Eliminar archivo",  button_style='warning')
load_button = widgets.Button(description="Cargar archivos",  button_style='primary')

add_button.on_click(add_file_input)
remove_button.on_click(remove_file_input)
load_button.on_click(process_files)

# Definir función para actualizar UI
def update_ui():
    clear_output()
    display(project_name)
    for path_input in file_paths:
        display(path_input)
    display(widgets.HBox([add_button, remove_button]))
    display(load_button)

# Inicializar UI con un campo de texto (input) para ruta de archivo
add_file_input()


### Previsualizar datos importados:

**Previsualizar tabla** con todos los registros importados:

In [ ]:
## Previsualizar dataframe con CSVs importados
display(df)
print(f"Filas/Columnas (shape) en registros importados: {df.shape}")

**Exportar copia en CSV con registros importados (o concatenados):**

In [ ]:
# Exportar archivo CSV con tabla de registros importados (y concatenados, en el caso de múltiples archivos)
df.to_csv(f"{project_name.value}_registros-importados.csv")

## **2. Limpieza y procesamient de registros importados**

### Generar identificadores únicos (IDs) por registro:

In [ ]:
# Definir función para asignar IDs únicos a cada fila en el data frame indicado como parámetro, comenzando desde '1000001'.
def assign_unique_ids(df):
    # Inicializar contador para IDs
    id_counter = 1000001

    # Crear copia de data frame original
    df_copy = df.copy()

    # Iterar a través de las filas del data frame
    for index, _ in enumerate(df_copy.index):
        # Dar formato a ID con ceros adicionales e incorporarlo al data frame
        formatted_id = str(id_counter).zfill(7)  # Se asegura de que sea un ID de 7 dígitos, agregando ceros cuando sea necesario
        df_copy.loc[index, 'id'] = formatted_id

        # Incrementar el contador del ID para la siguiente iteración
        id_counter += 1

    return df_copy

In [ ]:
# Ejecutar función para asignar IDs a cada registro y previsualizar tabla
if __name__ == "__main__":
    # Invocar la función con data frame de trabajo
    df_ids = assign_unique_ids(df)

# Sobreescribir data frame con nueva tabla con IDs generados
df = df_ids
df

### *Elegir columna y criterios para limpieza de texto sin aporte semántico:

In [ ]:
# Definir función para limpiar usuarios, hashtags y URLs
def limpiar_texto(texto, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado):
    # Eliminar usuarios si está activado
    if eliminar_usuarios:
        texto = re.sub(r"(?<!\w)@(\w+)(?!\w)", "", texto)

    # Eliminar hashtags si está activado
    if eliminar_hashtags:
        texto = re.sub(r"(?<!\w)#(\w+)(?!\w)", "", texto)

    # Eliminar URLs si está activado
    if eliminar_urls:
        texto = re.sub(r"(http|https|ftp)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", "", texto)
        texto = texto.lstrip(". ")

    # Aplicar regex personalizado si se proporciona
    if regex_personalizado:
        texto = re.sub(regex_personalizado, "", texto)

    return texto.strip()

# Definir función para agregar nueva columna con texto limpio (clean_text)
def agregarCleanTextADf(df, colText, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado):
    dfW = df.copy()
    dfW["clean_text"] = None

    for index, row in dfW.iterrows():
        text = str(row[colText])
        cleaned_text = limpiar_texto(text, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado)
        dfW.at[index, "clean_text"] = cleaned_text

    return dfW


# Crear widget de desplegable con columnas disponibles
available_columns = list(df.columns)
column_dropdown = widgets.Dropdown(options=available_columns, value=available_columns[0], description='Columna:')


# Definir widgets de IPyWidgets para la UI interactiva
eliminar_usuarios = widgets.Checkbox(value=True, description='Eliminar usuarios @')
eliminar_hashtags = widgets.Checkbox(value=True, description='Eliminar hashtags #')
eliminar_urls = widgets.Checkbox(value=True, description='Eliminar URLs')
regex_input = widgets.Text(value='', description='Otro (Regex):', placeholder='Escribe un regex opcional')


# Botón para ejecutar la limpieza
boton_limpiar = widgets.Button(description="Limpiar texto", button_style='warning')

# Caja de texto para mostrar el resultado
output_resultado = widgets.Output()

# Función que se ejecuta al hacer clic en el botón de limpiar texto
def ejecutar_limpieza(b):
    global dfCleanText  # Hacer que dfCleanText sea accesible globalmente
    with output_resultado:
        output_resultado.clear_output()  # Limpiar cualquier salida previa

        # Obtener el nombre de la columna seleccionada por el usuario
        text_column = column_dropdown.value

        # Verificar si la columna existe en el DataFrame
        if text_column not in df.columns:
            print(f"Error: La columna '{text_column}' no existe en el DataFrame.")
            return

        # Ejecutar la función de limpieza sobre el DataFrame seleccionado
        dfCleanText = agregarCleanTextADf(
            df, text_column,
            eliminar_usuarios.value,
            eliminar_hashtags.value,
            eliminar_urls.value,
            regex_input.value
        )

        # Mostrar el DataFrame con la nueva columna 'clean_text'
        print("Texto limpio aplicado. DataFrame actualizado globalmente como 'dfCleanText'.")
        display(dfCleanText)

# Conectar el botón con la función de limpieza
boton_limpiar.on_click(ejecutar_limpieza)

# Desplegar los widgets en pantalla
display(column_dropdown, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_input, boton_limpiar, output_resultado)


### \*Elegir idioma de palabras vacías (*stop words*) a eliminar:

Elegir **idioma de** (desde diccionarios de NLTK), **eliminar *stop words*** y **agregar columna sem_text** con texto depurado:

In [ ]:
# Definir función para eliminar stopwords y signos de puntuación
def delete_stopwords(texto, stopwords_list):
    # Tokenizar el texto
    tokens = nltk.word_tokenize(texto)

    # Eliminar signos de puntuación
    tokens = [token for token in tokens if token.isalnum()]

    # Eliminar stop words
    tokens = [token for token in tokens if token.lower() not in stopwords_list]

    # Convertir la lista de tokens a un string
    texto_limpio = " ".join(tokens)

    return texto_limpio.strip()

# Definir función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
def agregarSemTextADf(df, colText, stopwords_list):
    dfW = df.copy()
    dfW["sem_text"] = None

    for index, row in dfW.iterrows():
        text = row[colText]
        cleaned_text = delete_stopwords(text, stopwords_list)
        dfW.at[index, "sem_text"] = cleaned_text

    return dfW

# Lista de idiomas disponibles en NLTK
idiomas_stopwords = [
    'arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french',
    'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali',
    'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish',
    'swedish', 'tajik', 'turkish'
]

# Crear dropdown para seleccionar idioma
dropdown_idioma = widgets.Dropdown(
    options=idiomas_stopwords,
    value='spanish',  # Valor por defecto
    description='Idioma:'
)

# Botón para eliminar stopwords
boton_eliminar_stopwords = widgets.Button(description="Eliminar stopwords", button_style='warning')

# Output para mostrar el resultado
output_resultado_stopwords = widgets.Output()

# Función que se ejecuta al hacer clic en el botón de eliminar stopwords
def ejecutar_eliminacion_stopwords(b):
    global dfClean  # Hacer que dfClean sea accesible globalmente
    with output_resultado_stopwords:
        output_resultado_stopwords.clear_output()  # Limpiar cualquier salida previa

        # Obtener el idioma seleccionado
        idioma_seleccionado = dropdown_idioma.value

        # Descargar stopwords del idioma seleccionado
        stopwords_list = stopwords.words(idioma_seleccionado)

        # Verificar si la columna 'clean_text' existe en el DataFrame
        if 'clean_text' not in dfCleanText.columns:
            print("Error: La columna 'clean_text' no existe en el DataFrame.")
            return

        # Ejecutar la función para eliminar stopwords y agregar la nueva columna 'sem_text'
        dfClean = agregarSemTextADf(dfCleanText, 'clean_text', stopwords_list)

        # Mostrar el DataFrame con la nueva columna 'sem_text'
        print(f"Stopwords en {idioma_seleccionado} eliminadas. DataFrame actualizado globalmente como 'dfClean'.")
        display(dfClean)

# Conectar el botón con la función de eliminación de stopwords
boton_eliminar_stopwords.on_click(ejecutar_eliminacion_stopwords)

# Desplegar los widgets en pantalla
display(dropdown_idioma, boton_eliminar_stopwords, output_resultado_stopwords)

### Calcular valores promedio de audiencia, impresiones y costo:

Definir función para **extraer los valores numéricos** de rangos dados por Meta en campos de ***audience*, *impressions* y *spend***, y **promediarlos** (en caso de registrar un límite inferior y superior).

In [ ]:
# Sobreescribir df general con datos limpios
df = dfClean

In [ ]:
# Función para convertir una cadena de entrada en un diccionario
def string_to_dict(input_string):
    # Definir regex para encontrar diferentes formatos de límites en strings con rang
    patterns = [
        r'lower_bound: (\d+), upper_bound: (\d+)',
        r'lower_bound: (\d+)',
        r'upper_bound: (\d+)'
    ]

    # Inicializar un diccionario de resultados con valores por defecto de None para ambos límites
    result_dict = {'lower_bound': None, 'upper_bound': None}

    # Iterar sobre los patrones de Regex definidos:
    for pattern in patterns:
        # Buscar una coincidencia del patrón en string
        match = re.search(pattern, input_string)
        if match:
            # Si se encuentra una coincidencia, extraer el valor correspondiente del límite y almacenarlo en el diccionario de resultados
            if 'lower_bound' in pattern:
                result_dict['lower_bound'] = int(match.group(1))
            if 'upper_bound' in pattern:
                result_dict['upper_bound'] = int(match.group(1))
            break

    # Devolver el diccionario con los valores extraídos
    return result_dict

# Función para extraer diccionarios de string en bruto
def extract_dicts(raw_str):
    # Definir un patrón para encerrar cualquier contenido entre llaves
    pattern = r'\{.*?\}'
    # Buscar todas las coincidencias del patrón en la representación de cadena del DataFrame
    matches = re.findall(pattern, str(raw_str))
    # Convertir cada coincidencia extraída en un diccionario Python usando eval()
    dicts = [eval(match) for match in matches]

    # Devolver una lista de diccionarios extraídos
    return dicts

Ejecutar procesamiento de datos para procesar formatos de fechas y **obtener los siguientes nuevos campos**:

**Rangos y promedio de dinero** invertido en anuncio:
* spend_min
* spend_max
* spend_avg

**Rangos y promedio de impresiones** efectivas:
* impressions_min
* impressions_max
* impressions_avg

**Rangos y promedio de audiencia** potencial segmentada:
* audience_min
* audience_max
* audience_avg

**Duración** de campaña (en días):
* campaign_duration

**URLs exactas** de página y anuncio:
* page_url
* ad_url


In [ ]:
#Formato datetime
df['ad_creation_time'] = pd.to_datetime(df['ad_creation_time'])
df['ad_delivery_start_time'] = pd.to_datetime(df['ad_delivery_start_time'])
df['ad_delivery_stop_time'] = pd.to_datetime(df['ad_delivery_stop_time'])

#Transformar en diccionarios los datos de las columnas
df['spend_dict'] = df['spend'].apply(string_to_dict)
df['impressions_dict'] = df['impressions'].apply(string_to_dict)
df['estimated_audience_size_dict'] = df['estimated_audience_size'].apply(string_to_dict)

#Crear nuevas columnas
df['spend_min'] = df['spend_dict'].apply(lambda x: x['lower_bound'])
df['spend_max'] = df['spend_dict'].apply(lambda x: x['upper_bound'])
df['impressions_min'] = df['impressions_dict'].apply(lambda x: x['lower_bound'])
df['impressions_max'] = df['impressions_dict'].apply(lambda x: x['upper_bound'])
df['spend_avg'] = (df['spend_min'] + df['spend_max']) / 2
df['impressions_avg'] = (df['impressions_min'] + df['impressions_max']) / 2

# Extract audience_min and audience_max
df['audience_min'] = df['estimated_audience_size_dict'].apply(lambda x: x['lower_bound'])
df['audience_max'] = df['estimated_audience_size_dict'].apply(lambda x: x['upper_bound'])

# Calculate audience_avg
df['audience_avg'] = df.apply(lambda row: (row['audience_min'] + row['audience_max']) / 2 if pd.notna(row['audience_max']) else row['audience_min'], axis=1)



df['page_url'] = "https://www.facebook.com/ads/library/?id=" + df['page_id'].astype(str)
df['ad_url'] = "https://www.facebook.com/" + df['ad_archive_id'].astype(str)

df['campaign_duration'] = df['ad_delivery_stop_time'] - df['ad_delivery_start_time']
df['campaign_duration'] = df['campaign_duration'].dt.days

# Extraer diccionarios
df['demographic_distribution_dict'] = df['demographic_distribution'].apply(extract_dicts)
df['delivery_by_region_dict'] = df['delivery_by_region'].apply(extract_dicts)


In [ ]:
df.head(10)

In [ ]:
# Exportar archivo CSV con datos limpios y procesados
df.to_csv(f"{project_name.value}_procesados.csv")

print(f"{project_name.value}_procesados.csv")

## **3. Depuración de registros desde diccionarios personalizados (con términos de filtrado o descarte)**

### *Definir uso de de diccionario de depuración:

A partir de diccionarios personalizados con términos de descarte o filtración, elige depurar o mantener, respectivamente, registros que los contengan.

El diccionario debe cargarse en formato CSV y contener, al menos, los siguientes campos:

| palabra | tipo | categoría | diccionario |
|---------|------|-----------|-------------|
|         |      |           |             |
|         |      |           |             |
|         |      |           |             |

Ejemplo:

| palabra | tipo   | categoría   | diccionario        |
|---------|--------|-------------|--------------------|
| idiota  | ofensa | humillación | ofensa-humillación |
| zorra   | ofensa | género      | ofensa-género      |
|         |        |             |                    |

Puedes [encontrar aquí una copia del diccionario](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=sharing) con la estructura requerida, en formato CSV, para descargar, llenar e incorporar a este cuaderno de código.

**Nota:** La versión actual de este cuaderno de código permite elegir entre las siguientes opciones sobre la carga de diccionarios:


- ***No cargar diccionario:*** Ignorar esta funcionalidad y no usar diccionarios.
- ***Diccionario de descarte (negativo):*** Se eliminarán todos los registros que mencionen alguno de esos términos.
- ***Diccionario de filtrado (positivo):*** Se mantendrán solo los registros que mencionen alguno de esos términos.

In [ ]:
# Create the dropdown widget for dictionary usage selection
cargar_diccionario = widgets.Dropdown(
    options=["No cargar diccionario", "Diccionario de descarte (negativo)", "Diccionario de filtrado (positivo)"],
    value="No cargar diccionario",
    description="Elegir uso:"
)

# Create the text input widget for file path
rutaDicc = widgets.Text(
    value="",
    placeholder="Escribir ruta a archivo CSV",
    description="Ruta:"
)

# Create the "Aceptar" button
accept_button = widgets.Button(description="Aceptar")

# Create an output widget to display results or errors
output = widgets.Output()

# Function to load the dictionary based on user input
def load_dictionary(b):
    with output:
        clear_output()
        if cargar_diccionario.value != "No cargar diccionario":
            try:
                global dfDiccionario
                dfDiccionario = pd.read_csv(rutaDicc.value)
                print(f"Archivo de diccionario '{rutaDicc.value}' cargado exitosamente.")
                display(dfDiccionario.head())  # Show the first few rows of the loaded dictionary
            except FileNotFoundError:
                print(f"El archivo de diccionario '{rutaDicc.value}' no fue encontrado. Por favor, verifique la ruta.")
        else:
            dfDiccionario = None
            print("No se cargó ningún diccionario.")

# Link the button to the load function
accept_button.on_click(load_dictionary)

# Display the widgets
display(cargar_diccionario, rutaDicc, accept_button, output)


In [ ]:
# Verificar uso de diccionario elegido
print(f"Uso de diccionario elegido: {cargar_diccionario.value}")

### Aplicar filtrado por diccionario y previsualizar resultados (opcional, solo si se cargó algún diccionario):

**Previsualizar diccionario de descarte importado:**

In [ ]:
if cargar_diccionario.value != "No cargar diccionario":
  # Previsualizar tabla de diccionario cargado (en caso de haber elegido utilizar diccionarios)
  try:
      display(dfDiccionario.head())  # Previsualizar primeros registros de diccionario
      print(f"\nFilas y columnas en diccionario cargado:")  # Verificar número de filas y columnas en diccionario
      print(f"Shape: {dfDiccionario.shape}")  # Verificar número de filas y columnas en diccionario
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")


**Filtrar registros a partir de diccionario cargado y modalidad de depuración (positiva o negativa):**

In [ ]:
def filtrar_registros(df_registros, df_terminos, colTexto, cargar_diccionario):
    # Definir los términos del diccionario
    terminos = df_terminos["palabra"].tolist()
    # Compilar expresiones regulares una sola vez
    expresiones_regex = [
        re.compile(r"(?<!\S)?(?:\s|[.,;:?!¡¿]){}(?:\s|[.,;:?!¡¿])?(?!\S)".format(re.escape(termino)), re.IGNORECASE)
        for termino in terminos
    ]

    # Copiar el dataframe de registros y agregar columnas auxiliares
    df_registros_filtrados = df_registros.copy()
    df_registros_filtrados["contiene_termino"] = False
    df_registros_filtrados["razon_eliminacion"] = ""

    # Filtrar registros
    for i in range(df_registros_filtrados.shape[0]):
        texto = str(df_registros_filtrados.loc[i, colTexto]).lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

        # Buscar coincidencias con expresiones regulares
        for expresion, razon in zip(expresiones_regex, df_terminos["categoría"]):
            coincidencias = expresion.findall(f" {texto} ")
            if coincidencias:
                df_registros_filtrados.loc[i, "contiene_termino"] = True
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Presencia de términos relacionados a {razon}"
                break
            else:
                df_registros_filtrados.loc[i, "contiene_termino"] = False
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Ausencia de términos relacionados a {razon}"

    # Revisar uso elegido del diccionario, para descarte (negativo) o filtrado (positivo)
    if cargar_diccionario == "Diccionario de descarte (negativo)":
        # Mantener registros que no contengan términos del diccionario (negativo)
        df_registros_filtrados_final = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
    elif cargar_diccionario == "Diccionario de filtrado (positivo)":
        # Mantener registros que sí contengan al menos un término del diccionario (positivo)
        df_registros_filtrados_final = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]

    # Eliminar columnas auxiliares para tabla con registros depurados
    df_registros_filtrados_final = df_registros_filtrados_final.drop(columns=["contiene_termino", "razon_eliminacion"])

    df_registros_filtrados_final = df_registros_filtrados_final.reset_index(drop=True)
    df_registros_eliminados = df_registros_eliminados.reset_index(drop=True)

    return df_registros_filtrados_final, df_registros_eliminados


In [ ]:
# Definir el nombre de la columna que contiene el texto, si no está ya definido
# Asegúrate de que esta columna exista en el DataFrame dfClean
if 'text_column' not in globals():
    text_column = 'sem_text'  # Cambia 'sem_text' por el nombre correcto de la columna

# Ejecutar depuración de registros por diccionario cargado
if cargar_diccionario.value != "No cargar diccionario":
    try:
        # Asegúrate de que dfDiccionario esté definido y cargado antes de la función
        if 'dfDiccionario' in globals() and isinstance(dfDiccionario, pd.DataFrame):
            df_depurados_dicc, df_eliminados_dicc = filtrar_registros(dfClean, dfDiccionario, text_column, cargar_diccionario.value)
        else:
            print("El diccionario no está cargado o no es un DataFrame válido.")
    except NameError as e:
        print(f"Error: {e}")
else:
    print("No se cargó ningún diccionario...")


**Previsualizar registros depurados y eliminados (opcional):**

In [ ]:
# Previsualizar tabla de registros eliminados y verificar su número de filas y columnas
if cargar_diccionario.value != "No cargar diccionario":
  try:
    display(df_eliminados_dicc)
    print(f"\nFilas/Columnas (shape) en registros eliminados por diccionario: {df_eliminados_dicc.shape}")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Previsualizar data frame final
if cargar_diccionario.value != "No cargar diccionario":
    dfFinal = df_depurados_dicc
    dfFinal.to_csv(f"{project_name.value}_filtrados-dicc.csv")
    print(f"{project_name.value}_filtrados-dicc.csv")
else:
    dfFinal = dfClean

## **4. Análisis exploratorio de datos (visualizaciones de metadatos de anuncios y su segmentación de audiencias)**
*Se recomienda utilizar esta sección con conjuntos de datos no mayores a 20,000 filas.

### Resumen estadistico:

In [ ]:
# Seleccionar columnas numericas
columnas_numericas = ['spend_min', 'spend_max', 'impressions_min', 'impressions_max', 'audience_min', 'audience_max','spend_avg', 'audience_avg', 'impressions_avg', 'campaign_duration']
# columnas_numericas = ['spend_min', 'spend_max', 'impressions_min', 'impressions_max','spend_avg', 'impressions_avg', 'campaign_duration']
datos_numericos = dfFinal[columnas_numericas]

# Crear resumen
resumen_estadistico = datos_numericos.describe()

# Mostrar resultados
resumen_estadistico

### Distribución temporal de los anuncios:

In [ ]:
def plot_temporal_distribution(df, column, bins=30, kde=True, color='skyblue'):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], bins=bins, kde=kde, color=color)
    plt.title(f'Distribución Temporal de {column}')
    plt.xlabel('Fecha de Creación del Anuncio')
    plt.ylabel('Frecuencia')
    plt.show()

# Mostrar resultados
plot_temporal_distribution(dfFinal, 'ad_creation_time')
plot_temporal_distribution(dfFinal, 'campaign_duration')

### Análisis de audiencia segmentada, gasto e impresiones a través del tiempo:
(Puede tomar algo de tiempo...)

In [ ]:
def plot_lineplot(df, x_column, y_column, hue_column, title, x_label, y_label):
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=x_column, y=y_column, data=df, hue=hue_column)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(title=hue_column)
    plt.show()

# Mostrar resultados
plot_lineplot(dfFinal, 'ad_creation_time', 'audience_avg', 'page_name', 'Avg Audience Size a lo largo del Tiempo', 'Fecha de Creación del Anuncio', 'Avg Audience Size')
plot_lineplot(dfFinal, 'ad_creation_time', 'spend_avg', 'page_name', 'Avg Spend a lo largo del Tiempo', 'Fecha de Creación del Anuncio', 'Avg Spend')
plot_lineplot(dfFinal, 'ad_creation_time', 'impressions_avg', 'page_name', 'Avg Impressions a lo largo del Tiempo', 'Fecha de Creación del Anuncio', 'Avg Impressions')


### Costo promedio por impresion:

In [ ]:
# Calcular el CPI & CPM
cpi_promedio = dfFinal['spend_avg'].mean() / dfFinal['impressions_avg'].mean()
cpm_promedio = cpi_promedio * 1000

# Mostrar resultados
print('Costo por Impresion',cpi_promedio,'...', 'Costo por 1000 Impresiones', cpm_promedio)

### Matriz de Correlación:

In [ ]:
# Seleccionar columnas numericas
columnas_numericas = ['spend_min', 'impressions_min', 'audience_min', 'campaign_duration',
                      'spend_max', 'impressions_max', 'audience_max', 'campaign_duration',
                      'spend_avg', 'audience_avg', 'impressions_avg', 'campaign_duration']

# Calcular la matriz de correlación
correlation_matrix = dfFinal[columnas_numericas].corr()

# Mostrar resultados
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title("Matriz de Correlación entre Variables Numéricas")
plt.show()


### Analisis de Dispersion y Relacion entre Variables:

In [ ]:
def scatter_matrix(dataframe, numeric_columns):
    sns.pairplot(dataframe[numeric_columns])
    plt.suptitle("Relaciones entre Variables Numéricas", y=1.02)
    plt.show()

# Variables numericas de interes
# columnas_numericas_min = ['spend_min', 'impressions_min', 'campaign_duration']
# columnas_numericas_max = ['spend_max', 'impressions_max', 'campaign_duration']
# columnas_numericas_avg = ['spend_avg', 'impressions_avg', 'campaign_duration']


columnas_numericas_min = ['spend_min', 'impressions_min', 'audience_min', 'campaign_duration']
columnas_numericas_max = ['spend_max', 'impressions_max', 'audience_max', 'campaign_duration']
columnas_numericas_avg = ['spend_avg', 'audience_avg', 'impressions_avg', 'campaign_duration']

# Mostrar resultados
scatter_matrix(dfFinal, columnas_numericas_min)
scatter_matrix(dfFinal, columnas_numericas_max)
scatter_matrix(dfFinal, columnas_numericas_avg)

### Visualización de variables categóricas de anuncios:

In [ ]:
def plot_categorical_distribution(df, column, top_n=10, figsize=(12, 6)):
    plt.figure(figsize=figsize)
    top_categories = df[column].value_counts().nlargest(top_n)

    sns.barplot(x=top_categories.values, y=top_categories.index, hue=top_categories.index, palette='viridis', legend=False)

    plt.title(f'Distribución de {column}')
    plt.xlabel('Frecuencia')
    plt.ylabel(column)
    plt.show()

# Seleccionar columnas categoricas
columnas_categoricas = ['page_name', 'byline', 'ad_creative_link_titles', 'ad_creative_bodies',
                        'ad_creative_link_captions', 'ad_creative_link_descriptions',
                        'publisher_platforms', 'languages']

# Mostrar resultados
for column in columnas_categoricas:
    plot_categorical_distribution(dfFinal, column)


### Distribución normalizada de segmentación de audiencias de anuncios por Edad y Género:
(Puede tomar algo de tiempo...)


In [ ]:
def create_normalized_df(df, column_name, id_column='ad_archive_id', subcolumn_names=None):
    df_normalized = pd.DataFrame()

    for index, row in df.iterrows():
        for entry in row[column_name]:
            subcolumn_values = [entry[subcolumn] for subcolumn in subcolumn_names] if subcolumn_names else []
            percentage = entry['percentage']

            data = {id_column: [row[id_column]], **dict(zip(subcolumn_names, subcolumn_values)), 'percentage': [percentage]}
            df_normalized = pd.concat([df_normalized, pd.DataFrame(data)])

    total_percentage_by_group = df_normalized.groupby(subcolumn_names)['percentage'].sum()
    total_percentage_normalized = total_percentage_by_group / total_percentage_by_group.sum()

    df_normalized_final = pd.DataFrame({
        **{subcolumn: total_percentage_normalized.index.get_level_values(subcolumn) for subcolumn in subcolumn_names},
        'percentage': total_percentage_normalized.values * 100 # indicar porcentaje sobre 100%
    })

    return df_normalized_final


In [ ]:
# Crear dataframe con los datos expandidos
df_demographics_normalized = create_normalized_df(dfFinal, 'demographic_distribution_dict', subcolumn_names=['age', 'gender'])

# Crear tabla y mostrar
fig_normalized = px.treemap(df_demographics_normalized, path=['age','gender'], values='percentage',
                            title='Treemap de Distribución Demográfica Normalizada',
                            color='percentage',
                            color_continuous_scale='viridis')

fig_normalized.show()

In [ ]:
# Previsualizar tabla de porcentajes de distribución por región
df_sorted_demographics =df_demographics_normalized.sort_values(by='percentage', ascending=False)

df_sorted_demographics.head(30)

In [ ]:
# Descargar archivo de datos CSV con valores normalizados de distribución por región:
df_sorted_demographics.to_csv(f"{project_name.value}_demographics.csv", index=False)

print(f"{project_name.value}_demographics.csv descargado.")

### Distribución geográfica de anuncios (por Región):
(Puede tomar algo de tiempo...)

In [ ]:
def create_normalized_regions_df(df, column_name, id_column='ad_archive_id', subcolumn_names=None):
    df_normalized = pd.DataFrame()

    for index, row in df.iterrows():
        for entry in row[column_name]:
            subcolumn_values = [entry[subcolumn] for subcolumn in subcolumn_names] if subcolumn_names else []
            percentage = entry['percentage']

            data = {id_column: [row[id_column]], **dict(zip(subcolumn_names, subcolumn_values)), 'percentage': [percentage]}
            df_normalized = pd.concat([df_normalized, pd.DataFrame(data)])

    total_percentage_by_group = df_normalized.groupby(subcolumn_names)['percentage'].sum()
    total_percentage_normalized = total_percentage_by_group / total_percentage_by_group.sum()

    df_normalized_regions_final = pd.DataFrame({
        **{subcolumn: total_percentage_normalized.index.get_level_values(subcolumn) for subcolumn in subcolumn_names},
        'percentage': total_percentage_normalized.values*100 # indicar porcentaje sobre 100%
    })

    return df_normalized_regions_final


In [ ]:
# Crear dataframe con los datos expandidos por región
df_regions_normalized = create_normalized_regions_df(dfFinal, 'delivery_by_region_dict', subcolumn_names=['region'])

# Crear tabla y mostrar
fig_normalized_regions = px.treemap(df_regions_normalized, path=['region'], values='percentage',
                            title='Treemap de Distribución por Región',
                            color='percentage',
                            color_continuous_scale='viridis')

fig_normalized_regions.show()

In [ ]:
# Previsualizar tabla de porcentajes de distribución por región
df_sorted_regions = df_regions_normalized.sort_values(by='percentage', ascending=False)
df_sorted_regions.head(10)

In [ ]:
# Descargar archivo de datos CSV con valores normalizados de distribución por región:
df_sorted_regions.to_csv(f"{project_name.value}_regions.csv", index=False)

print(f"{project_name.value}_regions.csv descargado.")

### Desplegar mapa de regiones por país (trabajo en progreso, solo EEUU, México y España, por ahora):

In [ ]:
# Función para transformar nombres de regiones
def transform_region_name(region):
    # Quitar acentos y caracteres especiales
    no_accents = unidecode.unidecode(region)
    # Convertir a mayúsculas
    upper_case = no_accents.upper()
    return upper_case

# Crear nuevo dataframe
df_transformed = df_sorted_regions.copy()

# Apicar transformación a nueva columna
df_transformed['region_transformed'] = df_transformed['region'].apply(transform_region_name)

# Desplegar nuevo dataframe
df_transformed

In [ ]:
# US estados
url = 'https://raw.githubusercontent.com/signalab/postprocesador-redes/refs/heads/main/anexos/mapas/us-states.json'

# MX estados
# url = 'https://raw.githubusercontent.com/signalab/postprocesador-redes/refs/heads/main/anexos/mapas/mx-states.json'

# ES comunidades autónomas
# url = 'https://raw.githubusercontent.com/signalab/postprocesador-redes/refs/heads/main/anexos/mapas/es-ccaa.json'



# Fetch datos GeoJSON
response = requests.get(url)
geojson_data = response.json()

In [ ]:
# # Crear mapa base
m = folium.Map(location=[0, 0], zoom_start=3)

# # Añadir capa de mapa coroplético
folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=df_transformed,
    columns=['region', 'percentage'],
    key_on='feature.properties.name',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentaje de anuncios por Región',
    nan_fill_color='gray',  # Color for regions with no data
    nan_fill_opacity=0.7,
    bins=5,  # Número de binas para escala de color
    reset=True  # Reincializar mapa antes de añadir capa coroplética
).add_to(m)

# # Añadir control sobre capa
folium.LayerControl().add_to(m)

# # Desplega mapa
m

## **5. Referencias**

*  Bird, Steven, Edward Loper & Ewan Klein (2009).
Natural Language Processing with Python.  O'Reilly Media Inc.
* Kiss, T., & Strunk, J. (2006). Unsupervised Multilingual Sentence Boundary Detection. Computational Linguistics, 32(4), 485-525. https://doi.org/10.1162/coli.2006.32.4.485]
* Meta. (2024) Meta AdLibrary [software]. https://www.facebook.com/ads/library/
* Rieder, B. (2024) YouTube Data Tools [software]. Digital Methods Initiative. https://ytdt.digitalmethods.net/

*Programación asistida ocasionalmente con herramientas de IA Generativa: ChatGPT, Phind, HuggingChat y Perplexity

## **6. Créditos**

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
 José Luis Almendarez González y Diego Arredondo Ortiz.

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Noviembre, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.
